# INVENTA - Prioritization of natural extracts for chemical originality discovery


In [1]:
from __future__ import print_function
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")
import numpy as np
import pandas as pd
import sys 
import lineup_widget
import ipywidgets as widgets
from ipywidgets import *
!jupyter nbextension enable --py --sys-prefix lineup_widget


sys.path.append('../src')
sys.path.append('../gnps_postprocessing/src') 

from import_data import*
from process_data import *
from FC import *
from LC import *
from SC import *
from CC import *
from plot import *

from gnps_download_results import *
from consolidates_structures import *
from gnps_results_postprocess import *

Enabling notebook extension lineup_widget/extension...
      - Validating: OK


# Paths and parameters to define

In [16]:
#paths:

metadata_filename = '../data/Celastraceae_Set_metadata_pos.tsv'                      # The path where the metadata is placed
quantitative_data_filename = '../data/Celastraceae_pos_quant_id.csv'                    # The path where the quantitative table is placed
tima_results_filename = '../data/Celastraceae_pos_spectral_match_results_repond.tsv' # The path where the ISDB reponderated file is placed
vectorized_data_filename = '../data/Celastraceae_memomatrix.csv'                     # The path where the MEMO matrix is placed
canopus_npc_summary_filename = '../data/canopus_npc_summary.tsv'                     # The path where the SIRIUS-CANOPUS results file is placed
sirius_annotations_filename = '../data/compound_identifications.tsv'                 # The path where the SIRIUS results file is placed

job_id= 'df71854c6e644b979228d96b521a490b' #"yourjobidgoeshere"  '4c919fcbc83d487493a487012afb920a'#MZmine2   'df71854c6e644b979228d96b521a490b' #MZmine3_ID

# metadata headers

species_column = 'ATTRIBUTE_Species'
genus_column = 'ATTRIBUTE_Genus'
family_column = 'ATTRIBUTE_Family'
organe_column = 'ATTRIBUTE_Organe'
filename_header = 'filename'

# parameters for cleaning-up annotations from GNPS 

max_ppm_error = 5                 # min error in ppm to consider an annotation valable
shared_peaks = 4                 # min number of shared peaks between the MS2 experimental and MS2 from the database, to consider an annotation valable
min_cosine = 0.6                  # min cosine score to consider an annotation valable
ionisation_mode = 'pos'           # ionisation mode according to experimental conditions 'pos' or 'neg'
max_spec_charge = 2

#quantitative table
data_process_origin = 'MZMine3' #'MZMine2'
use_ion_identity= True #False

#Feature_component

min_specificity = 0.90            # minimun feature specificity to consider

## 
multiple_organism_parts = True  #True: the specificity is going to be considered as the sum of the 'max_parts_per_organism' shared in the samples.
max_parts_per_organism = 4       #max recurrence of the same organism species (for example: 5 samples, same species but different plant part) 

## inputs to use: 
isdb_annotations = True          #True: the tima_results_filename will be considered in the calculations
sirius_annotations = True        #True: the sirius_annotations_filename will be considered in the calculations

## cut-offs: 
min_score_final = 0.3             #cut-off filter for considering an isdb annotation valable. You must be extremenly carefull with this parameter, '0.0' as default.
min_ZodiacScore = 0.9             #cut-off filter for considering a sirius annotation valable. It is used in combination with min_ConfidenceScore.
min_ConfidenceScore= 0.25         #cut-off filter for considering a sirius annotation valable. '0.0' as default.

annotation_preference = 0          # Only Annotated features: '1' or  Only Not annotated features: '0'

#Literature_component

LC_component = True               # LC will be calculated

max_comp_reported_sp = 20          # max number of compounds reported at species level, more than this value, the plant is considered less interesting
max_comp_reported_g = 50         # max number of compounds reported at genus level,more than this value, the plant is considered less interesting
max_comp_reported_f = 500           # max number of compounds reported at genus level,more than this value, the plant is considered less interesting

#weight for each taxonomic level 
ws = 1
wg = 1
wf = 1 

#Similarity_component

SC_component = True                # SC will be calculated

#Class_component

CC_component =  True              # CC will be calculated
min_class_confidence = 0.8       #cut-off filter for considering a sirius class valable. It is used in combination with min_recurrence.
min_recurrence = 5               # minimum recurrence of a chemical class to consider it acceptable

#specify the weight to modulate each component 
w1 = 1           # 1 means the value itself is taken into account. A 0.5 means onle half of the calculated value is taken into account
w2 = 1
w3 = 1
w4 = 1


# Prepare input files

### Download and clean the GNPS results

In [3]:
df_annotations = gnps_download_results(job_id, output_folder ='../data/all_annotations', return_annotation_table=True)
#df_annotations.head()

# Consolidate structure identifiers
gnps_annotations_consolidated  = consolidate_and_convert_structures(df_annotations, prefix='', smiles='Smiles', inchi='INCHI')

# Filter GNPS annotations
gnps_annotations_filtered = gnps_filter_annotations(gnps_annotations_consolidated, 'Consol_InChI', ionisation_mode, max_ppm_error, min_cosine, shared_peaks, max_spec_charge)

# Generate annotation attributes
annot_gnps_df = get_gnps_annotations(get_molecular_formula_from_inchi(gnps_annotations_filtered, 'Consol_InChI'))
#annot_gnps_df.head(2)


This is the GNPS job link: https://gnps.ucsd.edu/ProteoSAFe/status.jsp?task=df71854c6e644b979228d96b521a490b


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 46.0M    0 46.0M    0     0  1708k      0 --:--:--  0:00:27 --:--:-- 2480k


GNPS job results were succesfully downloaded as: ../data/all_annotations.zip
GNPS job results were succesfully extracted into the folder: ../data/all_annotations
   FEATURE-BASED MOLECULAR NETWORKING job detected - Version > 28
      2493 spectral library annotations in the job.
      16138 nodes in the network (including single nodes).
Both SMILES and InChI were inputted
Converting SMILES to mol object
  Salt(s) deleted in       : CN(CCCNC(=O)C1CCCO1)C2=NC3=CC(=C(C=C3C(=N2)N)OC)OC.Cl
  Remaining residue        : COc1cc2nc(N(C)CCCNC(=O)C3CCCO3)nc(N)c2cc1OC
Succesfully converted to mol object: 1923
Exception to the parsing: 0
Not available: 571
Converting INCHI to mol object


RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ERROR: [08:37:01] ERROR: 
RDKit ER

Succesfully converted to mol object: 2004
Exception to the parsing: 0
Not available: 490
Consolidating the lists
Total mol object from the list 1 = 1923
Mol object consolidated from list 2 = 218
Consolidated structures = 2141
Converting mol objects to SMILES iso
Converting mol objects to SMILES
Converting mol objects to InChI
Converting mol objects to InChIKey
End
Initial number of annotations: 2494
Remaining after ionisation mode filtering: 2436
Remaining after max_ppm_error filtering: 2036
Remaining after min_cosine filtering: 2036
Remaining after number of shared_peaks filtering: 2036
Remaining after number of spectrum charge filtering: 2036
Initial number of annotations filtering: 2036
After carbon containing adducts filtering: 2036
Valid molecular formula: 1751


### Metadata table

In [4]:
metadata_df = pd.read_csv(metadata_filename, sep='\t')
metadata_df.head(3)

,filename,ATTRIBUTE_Code,ATTRIBUTE_Type,ATTRIBUTE_Family,ATTRIBUTE_Genus,ATTRIBUTE_Species,ATTRIBUTE_Organe,ATTRIBUTE_Broad_organ,ATTRIBUTE_Tissue,ATTRIBUTE_Subsystem
0,LQ-01-61-01_pos.mzXML,V107694,Sample,Celastraceae,Catha,Catha edulis,Leaves,photosynthetic,green tissue,aboveground
1,LQ-01-61-02_pos.mzXML,V107695,Sample,Celastraceae,Catha,Catha edulis,Stems,woody vegetative,woody tissue,aboveground
2,LQ-01-61-03_pos.mzXML,V107696,Sample,Celastraceae,Catha,Catha edulis,Roots,roots,root tissue,belowground


In [5]:
#if you need to create an unique identifier column like Species|part, use as model the followin line. IF the colum is PRESENT, then don't run it.
metadata_df['ATTRIBUTE_Sppart'] = metadata_df[species_column]+ "|" + metadata_df[organe_column].map(str)

In [6]:
col_id_unique = filename_header #'ATTRIBUTE_Sppart'  # filename_header   #column containing an unique identifier for each sample, like Species_plantpart, Species_solvent. It could be the filename

### Quantification table

In [7]:
#Quantitative table
quant_df = quant_table(quantitative_data_filename, data_process_origin, use_ion_identity)
quant_df.head(3)

,LQ-01-61-01_pos.mzXML,LQ-01-61-02_pos.mzXML,LQ-01-61-03_pos.mzXML,LQ-01-61-04_pos.mzXML,LQ-01-61-05_pos.mzXML,LQ-01-61-06_pos.mzXML,LQ-01-61-07_pos.mzXML,LQ-01-61-08_pos.mzXML,LQ-01-61-09_pos.mzXML,LQ-01-61-10_pos.mzXML,...,LQ-01-61-69_pos.mzXML,LQ-01-61-70_pos.mzXML,LQ-01-61-71_pos.mzXML,LQ-01-61-72_pos.mzXML,LQ-01-61-73_pos.mzXML,LQ-01-61-74_pos.mzXML,LQ-01-61-75_pos.mzXML,LQ-01-61-76_pos.mzXML,LQ-01-61-77_pos.mzXML,LQ-01-61-78_pos.mzXML
annotation network number,,,,,,,,,,,,,,,,,,,,,
0.0,0.001896,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.087568,0.000000
1.0,0.000000,0.00000,0.007044,0.000000,0.0,0.000000,0.000000,0.000000,0.377265,0.000000,...,0.000000,0.0,0.000000,0.0,0.041531,0.00000,0.000000,0.0,0.000000,0.000000
2.0,0.000000,0.01081,0.001208,0.002477,0.0,0.002341,0.019336,0.033384,0.000000,0.016455,...,0.004177,0.0,0.004424,0.0,0.005340,0.00769,0.004954,0.0,0.060946,0.007016


In [8]:
#if Ion Identity is used, it is necesary to recover the information for each correlation group
correlation_groups_df= correlation_groups(quantitative_data_filename, use_ion_identity)

#### Combined and remove experimental controls


In [9]:
# merge tables
full_df = full_data(metadata_df, quant_df, filename_header)
full_df.head(2)

,filename,ATTRIBUTE_Code,ATTRIBUTE_Type,ATTRIBUTE_Family,ATTRIBUTE_Genus,ATTRIBUTE_Species,ATTRIBUTE_Organe,ATTRIBUTE_Broad_organ,ATTRIBUTE_Tissue,ATTRIBUTE_Subsystem,...,9992x,9993x,9994x,9995x,9996x,9998x,9999x,999x,99x,9x
0,LQ-01-61-01_pos.mzXML,V107694,Sample,Celastraceae,Catha,Catha edulis,Leaves,photosynthetic,green tissue,aboveground,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,LQ-01-61-02_pos.mzXML,V107695,Sample,Celastraceae,Catha,Catha edulis,Stems,woody vegetative,woody tissue,aboveground,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
#erase all the blanks and QC's - Change the string as needed
list_of_strings_for_QC_Blank_filter = ['Blank', 'QC']
column_to_use_for_filtering = 'ATTRIBUTE_Type' #this information should be included in the metadata table

filtered_full_df = drop_samples_based_on_string(full_df,'full_df',list_of_strings_for_QC_Blank_filter, column_to_use_for_filtering)
filtered_metadata_df = drop_samples_based_on_string(metadata_df, 'metadata_df', list_of_strings_for_QC_Blank_filter, column_to_use_for_filtering)

(78, 13200)
(76, 13200)
(77, 11)
(76, 11)


##### Make a minimal table for further processing & apply filter by intensity 

In [11]:
reduced_df = reduce_df(filtered_full_df, filtered_metadata_df, col_id_unique).transpose()

In [12]:
#you can choose between a quantile filter (only the features higher than the third quantile (75%) ares kept):
reduced_df = features_filter(reduced_df, min_threshold=0.02)

In [13]:
#or a simple intensity filter, it removes the features less than X % of relative area (by default 20%). If you want you can combine boht filters
reduced_df = quantile_filter(reduced_df, quantile_threshold=0.75)

##### visualization of the distribution variation before and after filtering by sample 

In [48]:
drop_down = widgets.Dropdown(options=reduced_df.columns,description='Sample to plot', disabled=False)
def dropdown_handler(change):
    global sample
    print(change.new)
    sample = change.new  # This line isn't working
drop_down.observe(dropdown_handler, names='value')
display(drop_down)

Dropdown(description='Sample to plot', options=('LQ-01-61-01_pos.mzXML', 'LQ-01-61-02_pos.mzXML', 'LQ-01-61-03…

In [49]:
distribution_to_plot(sample, quant_df, reduced_df)

# Start calculation the diferent components

# Feature component (FC)

#### FC.1. Annotation Rate

In [50]:
annot_gnps_df = pd.read_csv('../data_out/annot_gnps_df_nos.tsv', sep='\t').drop(['Unnamed: 0'],axis=1)

In [17]:
#load annotations files
annot_is_df       = get_isdb_annotations(tima_results_filename, isdb_annotations)
annot_sirius_df   = get_sirius_annotations(sirius_annotations_filename, sirius_annotations) 

annotation_df = annotations(annot_is_df, annot_sirius_df, sirius_annotations, isdb_annotations, min_score_final, min_ConfidenceScore, min_ZodiacScore, correlation_groups_df, use_ion_identity)
annotation_df.head(2)


,annotation network number,annotation
0,0.0,0
1,1.0,0


#### FC.2. FC computation

In [18]:
FC = feature_component(quant_df, reduced_df, annotation_df, metadata_df, family_column, genus_column, species_column, col_id_unique, min_specificity, annotation_preference, filename_header, annot_sirius_df, sirius_annotations, annot_gnps_df, min_ZodiacScore, multiple_organism_parts, max_parts_per_organism, use_ion_identity)
FC.head()

,filename,ATTRIBUTE_Family,ATTRIBUTE_Genus,ATTRIBUTE_Species,initial_F,filtered_F,Total_SF,Total_SNAF,FS,FC
13,LQ-01-61-14_pos.mzXML,Celastraceae,Evonymopsis,Evonymopsis sp,1312,3.0,2.0,2.0,0.67,0.67
22,LQ-01-61-23_pos.mzXML,Celastraceae,Euonymus,Euonymus cochinchinensis,598,14.0,11.0,9.0,0.79,0.64
6,LQ-01-61-07_pos.mzXML,Celastraceae,Celastrus,Celastrus paniculatus,1389,67.0,48.0,42.0,0.72,0.63
2,LQ-01-61-03_pos.mzXML,Celastraceae,Catha,Catha edulis,1321,16.0,12.0,10.0,0.75,0.62
30,LQ-01-61-31_pos.mzXML,Celastraceae,Euonymus,Euonymus frigidus,1425,39.0,27.0,23.0,0.69,0.59


In [53]:
#Show the results in an interactive way
interact(selection_changed_FC, selection=lineup_widget.LineUpWidget(FC));

/home/quirosgu/anaconda3/envs/inventa/lib/python3.7/site-packages/jupyter_client/session.py:718: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant



interactive(children=(LineUpWidget(value=[], description='selection', layout=Layout(align_self='stretch', heig…

# Literature component (LC)


#### LC.1. LC computation

In [54]:
LC = literature_component(LC_component, metadata_df, filename_header, species_column, genus_column, family_column, max_comp_reported_sp, max_comp_reported_g, max_comp_reported_f, ws, wg, wf)
LC

,filename,ATTRIBUTE_Family,ATTRIBUTE_Genus,ATTRIBUTE_Species,Reported_comp_Species,Reported_comp_Genus,Reported_comp_Family,LC
0,LQ-01-61-01_pos.mzXML,Celastraceae,Catha,Catha edulis,126.0,126.0,6064.0,0.79052
1,LQ-01-61-02_pos.mzXML,Celastraceae,Catha,Catha edulis,126.0,126.0,6064.0,0.79052
2,LQ-01-61-03_pos.mzXML,Celastraceae,Catha,Catha edulis,126.0,126.0,6064.0,0.79052
3,LQ-01-61-04_pos.mzXML,Celastraceae,Catha,Catha edulis,126.0,126.0,6064.0,0.79052
4,LQ-01-61-05_pos.mzXML,Celastraceae,Celastrus,Celastrus orbiculatus,212.0,732.0,6064.0,0.62632
...,...,...,...,...,...,...,...,...
72,LQ-01-61-73_pos.mzXML,Celastraceae,Tripterygium,Tripterygium hypoglaucum,162.0,1353.0,6064.0,0.52712
73,LQ-01-61-74_pos.mzXML,Celastraceae,Tripterygium,Tripterygium wilfordii,1011.0,1353.0,6064.0,0.10262
74,LQ-01-61-75_pos.mzXML,Celastraceae,Tripterygium,Tripterygium wilfordii,1011.0,1353.0,6064.0,0.10262
75,LQ-01-61-76_pos.mzXML,0,0,0,0.0,0.0,0.0,1.00000


# Similarity component (SC)

#### SC.1. SC computation

In [55]:
metric_df = pd.read_csv(vectorized_data_filename, sep=',', encoding= 'unicode_escape')
SC = similarity_component(metric_df, SC_component)
SC

,filename,anomaly_IF,anomaly_LOF,anomaly_OCSVM,SC
0,LQ-01-61-08_pos.mzXML,1,1,1,0
1,LQ-01-61-09_pos.mzXML,1,-1,1,1
2,LQ-01-61-15_pos.mzXML,1,1,1,0
3,LQ-01-61-05_pos.mzXML,1,1,1,0
4,LQ-01-61-12_pos.mzXML,1,1,1,0
...,...,...,...,...,...
73,LQ-01-61-37_pos.mzXML,-1,1,1,1
74,LQ-01-61-73_pos.mzXML,1,1,1,0
75,LQ-01-61-74_pos.mzXML,1,1,1,0
76,LQ-01-61-55_pos.mzXML,1,1,1,0


#### SC.2. Visualizing the outliers

NOTE: depending on the size of the data set, projections could take a while... 

#### SC.2.1 PCoA + UMAP combined visualization

In [56]:
pcoa_umap_2d(
    matrix= metric_df,
    data = SC,
    metric= 'braycurtis', 
    filename_header=filename_header)

/home/quirosgu/anaconda3/envs/inventa/lib/python3.7/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning:

The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.007224834998204779 and the largest is 2.5161027047806597.



##### SC.2.2 optionally you can chosse to plot only UMAP, PCoA or PCoA 3d

In [57]:
#umap_2d(matrix = metric_df,
#       data = SC, 
#       metadata=metadata_df, 
#       filename_header =filename_header)

In [58]:
#pcoa_2d(
#    matrix= metric_df,
#    data = SC,
#    metric= 'braycurtis', #
#     filename_header=filename_header)

In [65]:
pcoa_3d(
    matrix= metric_df,
    data = SC,
    metric= 'braycurtis',
    filename_header=filename_header)

/home/quirosgu/anaconda3/envs/inventa/lib/python3.7/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning:

The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.007224834998204779 and the largest is 2.5161027047806597.



# Class component (SC)

### CC.1. Retrive and clean the predicted chemical classes from Sirius

In [60]:
#row ID is not anymore a int, because of the x.... treat differently somehow. Multiple annotation by group? create a set by group like before?

def class_component(df3, filename_header, species_column, genus_column,family_column, metadata_df, reduced_df, col_id_unique, min_class_confidence, min_recurrence, CC_component):
    """ function to compute the class component based on the possible presence of new chemical classes 
    Args:
        df3: canopus_sunmmary
        Returns:
        None
    """

    if CC_component == True:

        def top_ions(col_id_unique, reduced_df):

                """ function to compute the top species, top filename and top species/plant part for each ion 
            Args:
                df1 = reduced_df, table of with index on sp/part column and features only.
                df2 = quantitative.csv file, output from MZmine
                Returns:
                None
            """
            #computes the % for each feature
                dfA = reduced_df.copy()
                #dfA = dfA.transpose()
                dfA = dfA.div(dfA.sum(axis=1), axis=0)
                dfA.reset_index(inplace=True)
                dfA.rename(columns={'index': 'row ID'}, inplace=True)
                dfA.set_index('row ID', inplace=True)
                dfA = dfA.astype(float)
                dfA['Feature_specificity'] = dfA.apply(lambda s: s.abs().nlargest(1).sum(), axis=1)
                dfA.reset_index(inplace=True)
                #df1 = df1.drop([0], axis=1)
                dfA = dfA[['row ID', 'Feature_specificity']]
                dfA['row ID']=dfA['row ID'].astype(int)

                #computes the top filename for each ion 
                df2 = reduced_df.copy()
                df2 = df2.div(df2.sum(axis=1), axis=0)
                df2 = df2.copy()
                df2 = df2.astype(float)
                df2 = df2.apply(lambda s: s.abs().nlargest(1).index.tolist(), axis=1)
                df2 = df2.to_frame()
                df2['filename'] = pd.DataFrame(df2[0].values.tolist(), index= df2.index)
                df2 = df2.drop([0], axis=1)
                df2.reset_index(inplace=True)
                df2.rename(columns={'index': 'row ID'}, inplace=True)

                df = pd.merge(left=dfA,right=df2, how='left',on='row ID')

                if col_id_unique != 'filename':
                    #computes the top species/part for each feature 
                    df3 = reduced_df.copy()
                    df3 = df3.transpose()
                    df3 = df3.astype(float)
                    df3 = df3.apply(lambda s: s.abs().nlargest(1).index.tolist(), axis=1)
                    df3 = df3.to_frame()
                    df3[[col_id_unique]] = pd.DataFrame(df3[0].values.tolist(),index= df3.index)
                    df3 = df3.drop([0], axis=1)
                    df3.reset_index(inplace=True)
                    df3.rename(columns={'index': 'row ID'}, inplace=True)
                    df3['row ID'] = df3['row ID'].astype(int)
                    
                    #merge all the data 
                    df = pd.merge(left=df3, right=df, how='left', on='row ID')
                else: 
                    df
                return df
                
        df1 = top_ions(col_id_unique, reduced_df)
        df2 = metadata_df.copy()

        LotusDB = pd.read_csv('../data_loc/LotusDB_inhouse_metadata.csv',sep=',').dropna()

        # merge with top filename with iones 
        #df3['shared name'] = df3['name'].str.split('_').str[-1].astype(int) #use this line if you don't have a column with the name/shared name

        ## Retrieve classes predicted by Sirius         
        #the specificity_df is used to assign the main biological source to each feature. 
        df3.rename(columns={'name': 'shared name'}, inplace=True)
        df3 = pd.merge(left=df1[['row ID', filename_header]], right=df3[['shared name', 'class', 'classProbability']], how='left', left_on='row ID', right_on='shared name').dropna()
        df3.drop('shared name', axis=1, inplace=True)

        #filter based on min_class_probability
        df3.drop(df3[df3.classProbability > min_class_confidence].index, inplace=True)

        #calculare recurrence of each class 
        df4= df3[[filename_header, 'class']].groupby([filename_header,'class']).size().reset_index()
        df4.rename(columns={0: 'recurrence'}, inplace=True)

        df4 = df4[df4['recurrence'] >= min_recurrence].groupby(filename_header).agg(set)
        df4.drop ('recurrence', axis=1, inplace=True)
            
        df5 = pd.merge(left=df2[[filename_header, species_column]], right=df4, how='left', left_on=filename_header, right_on=filename_header).dropna()

        ## Retrieve classes reported in Lotus

        #create a set of species present in the metatada and reduce the lotus DB to it
        set_sp = set(df5[species_column].dropna())
        LotusDB= LotusDB[LotusDB['organism_taxonomy_09species'].isin(set_sp)]

        #retrieve the chemical classes associated to the species and genus
        df7 = LotusDB[['organism_taxonomy_09species', 'structure_taxonomy_npclassifier_03class']].groupby('organism_taxonomy_09species').agg(set)
        df7.rename(columns={'structure_taxonomy_npclassifier_03class': 'Chemical_class_reported_in_species'}, inplace=True)
        df8 = LotusDB[['organism_taxonomy_08genus', 'structure_taxonomy_npclassifier_03class']].groupby('organism_taxonomy_08genus').agg(set)
        df8.rename(columns={'structure_taxonomy_npclassifier_03class': 'Chemical_class_reported_in_genus'}, inplace=True)
        #merge into a single dataframe
        df9 = pd.merge(df2[[filename_header, species_column, genus_column, family_column]], df7, left_on= species_column, right_on='organism_taxonomy_09species', how='left')
        df9 = pd.merge(df9,df8, left_on= genus_column, right_on='organism_taxonomy_08genus', how='left')

        #obtain the difference between the predicted and reported compounds
        df = pd.merge(df5[[filename_header, 'class']],df9,on=filename_header, how='left').dropna()

        df['New_CC_in_sp'] = df["class"] - df["Chemical_class_reported_in_species"]  #check if the chemical classes from Sirius are reported in the species
        df['New_CC_in_genus'] = df["New_CC_in_sp"] - df["Chemical_class_reported_in_genus"]

        #change all the NaN with a string to indicate lack of reports in the literature 
        df['Chemical_class_reported_in_species'] = df['Chemical_class_reported_in_species'].fillna('nothing in DB')
        df['Chemical_class_reported_in_genus'] = df['Chemical_class_reported_in_genus'].fillna('nothing in DB')

        df['New_CC_in_sp'] = df['New_CC_in_sp'].fillna('nothing in DB')
        df['New_CC_in_genus'] = df['New_CC_in_genus'].fillna('nothing in DB')

        def is_empty(df):
            """ function to check if the sets are empty or not 
                Args:
                    df = Class component column CC 
                    Returns:
                    None
            """
            if df:
                return 1 # if the column is not empty then 1, something is new in the sp &/ genus
            else:
                return 0

        df['CC'] = df['New_CC_in_sp'].apply(is_empty)
        df['CC'] = df['CC'].fillna(1)

        #df = pd.merge(df2[[filename_header]], df,how= 'left', on=filename_header)
        df.to_csv('../data_out/CC_results.tsv', sep='\t')
        return df
        
    else:
        print ('No search was done because the Class component is not going to be calculated')


In [61]:
canopus_npc_df = get_canopus_pred_classes(canopus_npc_summary_filename, CC_component)
canopus_npc_df.head()

,name,directoryName,pathway,pathwayProbability,superclass,superclassProbability,class,classProbability,ClassyFirePrediction,ClassyFirePredictionProbability
0,1,/tmp/SIRIUS/2111_Celastraceae/Workspace_SIRIUS...,Terpenoids,0.995492,Sesquiterpenoids,0.944588,Agarofuran sesquiterpenoids,0.983273,Agarofurans,0.999909
1,11106,/tmp/SIRIUS/2111_Celastraceae/Workspace_SIRIUS...,Terpenoids,0.986169,Triterpenoids,0.884784,NaN,0.000000,12-hydroxysteroids,0.534021
2,11107,/tmp/SIRIUS/2111_Celastraceae/Workspace_SIRIUS...,Terpenoids,0.928775,Sesquiterpenoids,0.955813,Drimane sesquiterpenoids,0.833584,Dicarboxylic acids and derivatives,0.645973
3,11108,/tmp/SIRIUS/2111_Celastraceae/Workspace_SIRIUS...,Terpenoids,0.996414,Triterpenoids,0.896329,NaN,0.000000,Triterpenoids,0.554810
4,11109,/tmp/SIRIUS/2111_Celastraceae/Workspace_SIRIUS...,Terpenoids,0.970741,Triterpenoids,0.903207,Oleanane triterpenoids,0.838534,Triterpenoids,0.979062


In [62]:
canopus_npc_df = get_canopus_pred_classes(canopus_npc_summary_filename, CC_component)
CC = class_component(canopus_npc_df, filename_header, species_column, genus_column,family_column, metadata_df, reduced_df, col_id_unique, min_class_confidence, min_recurrence, CC_component)
CC  

ValueError: invalid literal for int() with base 10: '10000x'

# Priority rank Results

In [63]:
PR = priority_rank(FC, LC, SC, CC, LC_component, SC_component, CC_component, w1, w2, w3, w4)
PR.head(10)


,filename,ATTRIBUTE_Family,ATTRIBUTE_Genus,ATTRIBUTE_Species,initial_F,filtered_F,Total_SF,Total_SNAF,FS,FC,LC,Reported_comp_Species,Reported_comp_Genus,Reported_comp_Family,SC,CC,New_CC_in_sp,New_CC_in_genus,PR
0,LQ-01-61-14_pos.mzXML,Celastraceae,Evonymopsis,Evonymopsis sp,1312,3.0,2.0,2.0,0.67,0.67,0.87872,0.0,0.0,6064.0,0,NaN,NaN,NaN,NaN
1,LQ-01-61-23_pos.mzXML,Celastraceae,Euonymus,Euonymus cochinchinensis,598,14.0,11.0,9.0,0.79,0.64,0.79072,0.0,440.0,6064.0,1,NaN,NaN,NaN,NaN
2,LQ-01-61-07_pos.mzXML,Celastraceae,Celastrus,Celastrus paniculatus,1389,67.0,48.0,42.0,0.72,0.63,0.69682,71.0,732.0,6064.0,1,NaN,NaN,NaN,NaN
3,LQ-01-61-03_pos.mzXML,Celastraceae,Catha,Catha edulis,1321,16.0,12.0,10.0,0.75,0.62,0.79052,126.0,126.0,6064.0,0,NaN,NaN,NaN,NaN
4,LQ-01-61-31_pos.mzXML,Celastraceae,Euonymus,Euonymus frigidus,1425,39.0,27.0,23.0,0.69,0.59,0.79072,0.0,440.0,6064.0,0,NaN,NaN,NaN,NaN
5,LQ-01-61-72_pos.mzXML,Celastraceae,Salacia,Salacia letestuana,902,110.0,88.0,64.0,0.80,0.58,0.77592,0.0,514.0,6064.0,1,NaN,NaN,NaN,NaN
6,LQ-01-61-56_pos.mzXML,Celastraceae,Maytenus,Maytenus undata,1506,39.0,29.0,22.0,0.74,0.56,0.80412,14.0,338.0,6064.0,0,NaN,NaN,NaN,NaN
7,LQ-01-61-37_pos.mzXML,Celastraceae,Euonymus,Euonymus sanguineus,1655,12.0,11.0,6.0,0.92,0.50,0.79022,1.0,440.0,6064.0,1,NaN,NaN,NaN,NaN
8,LQ-01-61-73_pos.mzXML,Celastraceae,Tripterygium,Tripterygium hypoglaucum,1192,46.0,27.0,23.0,0.59,0.50,0.52712,162.0,1353.0,6064.0,0,NaN,NaN,NaN,NaN
9,LQ-01-61-42_pos.mzXML,Celastraceae,Loeseneriella,Loeseneriella concinna,716,19.0,9.0,9.0,0.47,0.47,0.87872,0.0,0.0,6064.0,0,NaN,NaN,NaN,NaN


In [ ]:
#Cyt_format_visualization = Cyt_format(col_id_unique)

### Display results

In [64]:
#Show the results in an interactive way
#def selection_changed(selection):
#    return df.iloc[selection]
interact(selection_changed, selection=lineup_widget.LineUpWidget(PR));

/home/quirosgu/anaconda3/envs/inventa/lib/python3.7/site-packages/jupyter_client/session.py:718: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant



interactive(children=(LineUpWidget(value=[], description='selection', layout=Layout(align_self='stretch', heig…

#### Select the sample you wan to plot 

In [ ]:
drop_down = widgets.Dropdown(options=quant_df.columns,description='Sample to plot', disabled=False)
def dropdown_handler(change):
    global sample
    print(change.new)
    sample = change.new 
drop_down.observe(dropdown_handler, names='value')
display(drop_down) 

Dropdown(description='Sample to plot', options=('LQ-01-61-01_pos.mzXML', 'LQ-01-61-02_pos.mzXML', 'LQ-01-61-03…

#### Plot the selected sample

In [66]:
pseudochromatogram(sample, quantitative_data_filename, annotation_df, metadata_df, reduced_df, min_specificity, annotation_preference, species_column, organe_column, CC_component, canopus_npc_summary_filename, min_class_confidence, sirius_annotations, sirius_annotations_filename, min_ConfidenceScore, min_ZodiacScore, use_ion_identity, correlation_groups_df, data_process_origin, filename_header)
chromatogram2D(sample, quantitative_data_filename, annotation_df, metadata_df, reduced_df, min_specificity, annotation_preference, species_column, organe_column, CC_component, canopus_npc_summary_filename, min_class_confidence, sirius_annotations, sirius_annotations_filename, min_ConfidenceScore, min_ZodiacScore, use_ion_identity, correlation_groups_df, data_process_origin, filename_header)